In [20]:
import os
import numpy as np
from dfply import *
import pandas as pd
from datetime import datetime

In [21]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format

In [47]:
# 데이터 불러오기
자동차_상품군정보 = pd.read_excel('./data/자동차_현행추정부채/자동차_상품군정보.xlsx', sheet_name='상품군분류', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_분류 = pd.read_excel('./data/자동차_현행추정부채/자동차_상품군정보.xlsx', sheet_name='KICS분류')
금감원_금리시나리오 = pd.read_excel('./data/자동차_현행추정부채/금감원_금리시나리오.xlsx') # 상승/하락/평행/경사 구분코드 넣기
자동차_원수_개별추산액 = pd.read_excel('./data/자동차_현행추정부채/자동차_원수_개별추산액.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_원수_미경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_원수_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_원수_직전1년경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_원수_직전1년경과보험료.xlsx', dtype={'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_납입예정보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_납입예정보험료.xlsx', dtype={'CLG_YM': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'PYM_EPCT_YM': str})
자동차_출재_비례_미경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비례_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_출재_비례_개별추산액 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비례_개별추산액.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
자동차_출재_비비례_개별추산액 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비비례_개별추산액.xlsx', dtype={'CLG_YM': str})
자동차_출재_비례_직전1년경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비례_직전1년경과보험료.xlsx', dtype={'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDGR_CD': str, 'PDC_CD': str, 'BSC_CVR_CD': str})
자동차_출재_비비례_직전1년경과보험료 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_비비례_직전1년경과보험료.xlsx', dtype={'CLG_YM': str, 'BSC_CVR_CD': str})
자동차_진전추이 = pd.read_excel('./data/자동차_현행추정부채/자동차_진전추이.xlsx', dtype={'BSE_YM': str, 'AY_YM': str})
자동차_원수_손해조사비율 = pd.read_excel('./data/자동차_현행추정부채/자동차_원수_손해조사비율.xlsx', dtype={'CLG_YM': str, 'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_출재_손해조사비율 = pd.read_excel('./data/자동차_현행추정부채/자동차_출재_손해조사비율.xlsx', dtype={'CLG_YM': str, 'PDGR_CD': str, 'BSC_CVR_CD': str})
IBNR율 = pd.read_excel('./data/자동차_현행추정부채/IBNR율.xlsx', dtype={'ASUM_INPD_DVCD': str, 'RRNR_DVCD': str})
회수율 = pd.read_excel('./data/현행추정부채/회수율.xlsx', dtype={'ASUM_INPD_DVCD': str})
선도부도율 = pd.read_excel('./data/현행추정부채/선도부도율.xlsx', dtype={'BASE_YYMM': str, 'CRD_GRD_CD': str})
신용등급분류 = pd.read_excel('./data/현행추정부채/신용등급분류.xlsx', dtype={'CRD_GRD_CD': str})
재보험자_국내신용등급 = pd.read_excel('./data/현행추정부채/재보험자_국내신용등급.xlsx', dtype={'T02_RN_RINSC_CD': str})

In [48]:
# IBNR 가공
자동차_IBNR율 = IBNR율 >> mask(X.ASUM_INPD_DVCD == '2')
자동차_원수_IBNR율 = 자동차_IBNR율 >> mask(X.RRNR_DVCD=='01')
자동차_출재_IBNR율 = 자동차_IBNR율 >> mask(X.RRNR_DVCD=='03')

#### 1. 자동차/원수/계약 - 미경과보험료

In [49]:
# 자동차 PDGR_NM 가공
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료.merge(자동차_원수_IBNR율[['KEY', 'IBNR_RT']], left_on='BSC_CVR_CD', right_on='KEY', how='left') >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(IBNR_RT=if_else(X.IBNR_RT.isna(), 0, X.IBNR_RT)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> mutate(IBNR=X.OGL_URND_PRM*X.IBNR_RT)

In [50]:
 자동차_원수_미경과보험료_집계 = 자동차_원수_미경과보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(OGL_URND_PRM=np.sum(X.OGL_URND_PRM)) \
    >> mutate(OGL_URND_PRM=X.OGL_URND_PRM/1e3)
자동차_원수_미경과보험료_집계 = 자동차_원수_미경과보험료_집계.reset_index(drop=True)
자동차_원수_미경과보험료_집계

,PDGR_NM,OGL_URND_PRM
0,개인용자동차(물담보),"711,247,076.5400"
1,개인용자동차(인담보),"425,354,486.8800"
2,업무용자동차(물담보),"210,669,118.5900"
3,업무용자동차(인담보),"142,884,079.4100"
4,영업용자동차(물담보),"12,181,875.2500"
5,영업용자동차(인담보),"48,234,053.3900"
6,자동차기타,"78,921,776.5700"


#### 1-2. 자동차/원수/보상 - IBNR

In [51]:
자동차_원수_IBNR_집계 = 자동차_원수_미경과보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(IBNR=np.sum(X.IBNR)) \
    >> mutate(IBNR=X.IBNR/1e3)
자동차_원수_IBNR_집계 = 자동차_원수_IBNR_집계.reset_index(drop=True)
자동차_원수_IBNR_집계

,PDGR_NM,IBNR
0,개인용자동차(물담보),"13,551,736.6004"
1,개인용자동차(인담보),"33,908,319.6702"
2,업무용자동차(물담보),"4,631,841.7113"
3,업무용자동차(인담보),"11,888,468.7768"
4,영업용자동차(물담보),"310,985.3063"
5,영업용자동차(인담보),"3,988,382.1707"
6,자동차기타,"3,103,085.4870"


#### 2. 자동차/출재(비례)/계약 - 미경과보험료

In [52]:
# 자동차 PDGR_NM 가공
자동차_출재_비례_미경과보험료_가공 = 자동차_출재_비례_미경과보험료.merge(자동차_출재_IBNR율[['KEY', 'IBNR_RT']], left_on='BSC_CVR_CD', right_on='KEY', how='left') >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(T02_RN_RINSC_CD='124012') \
    >> left_join(재보험자_국내신용등급, on='T02_RN_RINSC_CD') \
    >> mutate(IBNR_RT=if_else(X.IBNR_RT.isna(), 0, X.IBNR_RT)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> mutate(IBNR=X.RN_URND_PRM*X.IBNR_RT)

In [53]:
# 자동차 출재_비례 계약 미경과보험료 집계
자동차_출재_비례_미경과보험료_집계 = 자동차_출재_비례_미경과보험료_가공 >> group_by(X.GRADE, X.PDGR_NM) \
    >> summarise(RN_URND_PRM=np.sum(X.RN_URND_PRM)) \
    >> mutate(RN_URND_PRM=X.RN_URND_PRM/1e3)
자동차_출재_비례_미경과보험료_집계 = 자동차_출재_비례_미경과보험료_집계.reset_index(drop=True)
자동차_출재_비례_미경과보험료_집계

,PDGR_NM,GRADE,RN_URND_PRM
0,개인용자동차(물담보),AA,"11,163.8780"
1,개인용자동차(인담보),AA,"5,881.2490"
2,업무용자동차(물담보),AA,"5,884,417.3920"
3,업무용자동차(인담보),AA,"4,139,076.3870"
4,자동차기타,AA,0.0000


In [54]:
자동차_출재_비례_IBNR_집계 = 자동차_출재_비례_미경과보험료_가공 >> group_by(X.PDGR_NM, X.GRADE) \
    >> summarise(IBNR=np.sum(X.IBNR)) \
    >> mutate(IBNR=X.IBNR/1e3)
자동차_출재_비례_IBNR_집계 = 자동차_출재_비례_IBNR_집계.reset_index(drop=True)
자동차_출재_비례_IBNR_집계

,GRADE,PDGR_NM,IBNR
0,AA,개인용자동차(물담보),193.1721
1,AA,개인용자동차(인담보),468.1886
2,AA,업무용자동차(물담보),"137,930.8726"
3,AA,업무용자동차(인담보),"346,904.8288"
4,AA,자동차기타,0.0000


#### 2-2. 자동차/출재(비례)/보상 - IBNR

In [55]:
자동차_원수_IBNR_집계 = 자동차_원수_미경과보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(IBNR=np.sum(X.IBNR)) \
    >> mutate(IBNR=X.IBNR/1e3)
자동차_원수_IBNR_집계 = 자동차_원수_IBNR_집계.reset_index(drop=True)
자동차_원수_IBNR_집계

,PDGR_NM,IBNR
0,개인용자동차(물담보),"13,551,736.6004"
1,개인용자동차(인담보),"33,908,319.6702"
2,업무용자동차(물담보),"4,631,841.7113"
3,업무용자동차(인담보),"11,888,468.7768"
4,영업용자동차(물담보),"310,985.3063"
5,영업용자동차(인담보),"3,988,382.1707"
6,자동차기타,"3,103,085.4870"


#### 3. 할인율

In [56]:
금감원_금리시나리오_가공 = 금감원_금리시나리오 \
    >> mutate(MATURITY=X.BUCKET_END_DATE.dt.year*12 + X.BUCKET_END_DATE.dt.month - X.AS_OF_DATE.dt.year*12 - X.AS_OF_DATE.dt.month) \
    >> mutate(DISCOUNT=1/(1+X.VALUE)**(X.MATURITY/12)) \
    >> select(X.MATURITY, X.DISCOUNT)

#### 4. 자동차/원수/계약 - 미래보험료

In [57]:
# 자동차 계약 납입예정보험료 MATURITY 가공
자동차_납입예정보험료_가공 = 자동차_납입예정보험료 >> left_join(자동차_상품군정보, on=['PDGR_CD', 'BSC_CVR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> mutate(PYM_EPCT_YM=pd.to_datetime(자동차_납입예정보험료['PYM_EPCT_YM'], format='%Y%m')) \
    >> mutate(BSE_YM=pd.to_datetime(자동차_납입예정보험료['CLG_YM'], format='%Y%m')) \
    >> mutate(MATURITY=X.PYM_EPCT_YM.dt.year*12 + X.PYM_EPCT_YM.dt.month - X.BSE_YM.dt.year*12 - X.BSE_YM.dt.month) \
    >> drop(X.BSE_YM, X.PYM_EPCT_YM) \
    >> mutate(MATURITY=if_else(X.MATURITY < 0, 0, X.MATURITY))

# 자동차 할인율 반영
자동차_납입예정보험료_가공 = 자동차_납입예정보험료_가공 >> left_join(금감원_금리시나리오_가공, on='MATURITY') \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.EPCT_PRM*X.DISCOUNT) \
    >> mutate(WNCR_CNV_RN_PRM_DISC=X.RN_PRM*X.DISCOUNT)

In [58]:
# 자동차 계약 납입예정보험료 집계
자동차_납입예정보험료_집계 = 자동차_납입예정보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(WNCR_CNV_EPCT_PRM_DISC=np.sum(X.WNCR_CNV_EPCT_PRM_DISC), WNCR_CNV_RN_PRM_DISC=np.sum(X.WNCR_CNV_RN_PRM_DISC)) \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.WNCR_CNV_EPCT_PRM_DISC/1e3, WNCR_CNV_RN_PRM_DISC=X.WNCR_CNV_RN_PRM_DISC/1e3)
자동차_납입예정보험료_집계 = 자동차_납입예정보험료_집계.reset_index(drop=True)
자동차_납입예정보험료_집계

,PDGR_NM,WNCR_CNV_EPCT_PRM_DISC,WNCR_CNV_RN_PRM_DISC
0,개인용자동차(물담보),"1,988,314.1371",0.0000
1,개인용자동차(인담보),"1,415,500.5618",0.0000
2,업무용자동차(물담보),"472,686.6435","13,238.2028"
3,업무용자동차(인담보),"501,941.2784","16,767.5130"
4,영업용자동차(물담보),"3,370,411.5899",0.0000
5,영업용자동차(인담보),"7,594,402.9080",0.0000
6,자동차기타,"10,409,946.6590",0.0000


#### 5. 자동차/원수/보상 - 개별추산액

In [59]:
자동차_회수율 = 회수율 >> mask(X.ASUM_INPD_DVCD == '2') \
    >> mask(X.KEY.str.slice(-2) != 'ZZ') \
    >> drop(X.LAST_MODIFIED_BY, X.LAST_UPDATE_DATE) \
    >> mutate(RRNR_DVCD=X.KEY.str.slice(3,5)) \
    >> mutate(BSC_CVR_CD=X.KEY.str.slice(-2)) \

자동차_원수_회수율 = 자동차_회수율 >> mask(X.RRNR_DVCD=='01')
자동차_출재_회수율 = 자동차_회수율 >> mask(X.RRNR_DVCD=='03')

In [60]:
# 자동차 PDGR_NM 가공
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_원수_회수율[['BSC_CVR_CD', 'RCVR_RT']], on='BSC_CVR_CD') \
    >> mutate(RCVR_RT=if_else(X.RCVR_RT.isna(), 0, X.RCVR_RT)) \
    >> mutate(OST_AMT_RCVR=X.OST_AMT*(1-X.RCVR_RT))

In [61]:
자동차_원수_개별추산액_집계 = 자동차_원수_개별추산액_가공 >> group_by(X.PDGR_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT), OST_AMT_RCVR=np.sum(X.OST_AMT_RCVR)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3, OST_AMT_RCVR=X.OST_AMT_RCVR/1e3)
자동차_원수_개별추산액_집계 = 자동차_원수_개별추산액_집계.reset_index(drop=True)
자동차_원수_개별추산액_집계

,PDGR_NM,OST_AMT,OST_AMT_RCVR
0,개인용자동차(물담보),"71,221,844.2290","68,162,465.3880"
1,개인용자동차(인담보),"261,346,646.7910","247,763,891.3681"
2,업무용자동차(물담보),"22,135,577.5880","21,306,645.6942"
3,업무용자동차(인담보),"108,916,198.8020","105,398,628.6437"
4,영업용자동차(물담보),"2,290,806.0580","2,239,456.1724"
5,영업용자동차(인담보),"46,671,633.3280","45,754,655.6677"
6,자동차기타,"18,046,375.7240","17,629,170.9908"


#### 6. 자동차/출재(비례)/보상 - 개별추산액

In [62]:
# 자동차 출재 보상 PDGR_NM 가공
자동차_출재_비례_개별추산액_가공 = 자동차_출재_비례_개별추산액 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM)) \
    >> left_join(자동차_출재_회수율[['BSC_CVR_CD', 'RCVR_RT']], on='BSC_CVR_CD') \
    >> mutate(RCVR_RT=if_else(X.RCVR_RT.isna(), 0, X.RCVR_RT)) \
    >> mutate(RN_OST_AMT_RCVR=X.RN_OST_AMT*(1-X.RCVR_RT))

In [63]:
# 자동차 출재 비례 보상 개별추산액 집계
자동차_출재_비례_개별추산액_집계 = 자동차_출재_비례_개별추산액_가공 >> group_by(X.PDGR_NM) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT), RN_OST_AMT_RCVR=np.sum(X.RN_OST_AMT_RCVR)) \
    >> mutate(RN_OST_AMT=X.RN_OST_AMT/1e3, RN_OST_AMT_RCVR=X.RN_OST_AMT_RCVR/1e3)
자동차_출재_비례_개별추산액_집계 = 자동차_출재_비례_개별추산액_집계.reset_index(drop=True)
자동차_출재_비례_개별추산액_집계

,PDGR_NM,RN_OST_AMT,RN_OST_AMT_RCVR
0,개인용자동차(물담보),"125,784.7020","120,283.6274"
1,개인용자동차(인담보),"991,197.6270","957,115.1103"
2,업무용자동차(물담보),"596,688.3590","576,222.4301"
3,업무용자동차(인담보),"1,122,283.3010","1,092,542.8641"
4,영업용자동차(인담보),"50,144.8440","50,011.3500"
5,자동차기타,"533,480.6690","531,166.0540"


In [64]:
# 자동차 출재 비비례 보상 개별추산액 집계
자동차_출재_비비례_개별추산액_집계 = 자동차_출재_비비례_개별추산액 >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT).astype(float)) \
    >> mutate(RN_OST_AMT=X.RN_OST_AMT/1e3)
자동차_출재_비비례_개별추산액_집계

,RN_OST_AMT
0,"1,099,060.8430"


#### 6-2. 자동차/원수/보상 - 손해조사비율

In [65]:
# 자동차 출재 보상 PDGR_NM 가공
자동차_원수_손해조사비율_가공 = 자동차_원수_손해조사비율 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

In [66]:
# 자동차 출재 비례 보상 손해조사비율 집계
자동차_원수_손해조사비율_집계 = 자동차_원수_손해조사비율_가공 >> group_by(X.PDGR_NM) \
    >> summarise(PYN_BNF=np.sum(X.PYN_BNF), ACCD_IVMT=np.sum(X.ACCD_IVMT)) \
    >> mutate(PYN_BNF=X.PYN_BNF/1e3, ACCD_IVMT=X.ACCD_IVMT/1e3)
자동차_원수_손해조사비율_집계 = 자동차_원수_손해조사비율_집계.reset_index(drop=True)
자동차_원수_손해조사비율_집계

,PDGR_NM,PYN_BNF,ACCD_IVMT
0,개인용자동차(물담보),"2,662,046,618.4360","1,302,875.8240"
1,개인용자동차(인담보),"1,663,030,353.3120","744,407.9740"
2,업무용자동차(물담보),"938,938,982.8110","564,590.7430"
3,업무용자동차(인담보),"628,045,719.7880","243,929.9690"
4,영업용자동차(물담보),"92,016,778.7940","160,464.4540"
5,영업용자동차(인담보),"236,896,295.0810","81,945.4460"
6,자동차기타,"330,987,196.3520","355,782.9860"


In [67]:
# 자동차 출재 보상 PDGR_NM 가공
자동차_출재_손해조사비율_가공 = 자동차_출재_손해조사비율 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

#### 7. 할인요소

In [88]:
자동차_진전추이_가공 = 자동차_진전추이.copy()

할인요소 = []
for AT_PDGR_CD in 자동차_진전추이_가공['AT_PDGR_CD'].unique():
    자동차_진전추이_보종 = 자동차_진전추이_가공.query('AT_PDGR_CD == @AT_PDGR_CD').sort_values(by='AY_YM')
    할인요소_보종 = []
    m = len(자동차_진전추이_보종) - 1
    for i in range(m):
        n = m - i
        지급보험금 = 0
        for j in range(n):
            지급보험금 += 자동차_진전추이_보종.iloc[m-j, 3+j+1+i] - 자동차_진전추이_보종.iloc[m-j, 3+j+i]
        YEAR = i + 1
        평균지급기간 = i + 0.5
        지급보험금 = max(지급보험금, 0)
        할인요소_보종.append([YEAR, 평균지급기간, 지급보험금])
    할인요소_보종 = pd.DataFrame(할인요소_보종, columns=['YEAR', '준비금_평균지급기간', '지급보험금'])
    할인요소_보종['지급보험금_비중'] = 할인요소_보종['지급보험금']/할인요소_보종['지급보험금'].sum()
    할인요소_보종['누적비율'] = 할인요소_보종['지급보험금_비중'].cumsum()
    할인요소_보종['MATURITY'] = (할인요소_보종['준비금_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소_보종.insert(0, 'AT_PDGR_CD', AT_PDGR_CD)
    할인요소_보종['보험료_평균지급기간'] = 할인요소_보종['준비금_평균지급기간'] - 0.2071
    할인요소_보종['조정비율'] = 0
    for i in range(m):
        if i==m-1:
            할인요소_보종['조정비율'].iloc[i] = 1
        else:
            할인요소_보종['조정비율'].iloc[i] = 할인요소_보종['누적비율'].iloc[i]+0.2071*(할인요소_보종['누적비율'].iloc[i+1]-할인요소_보종['누적비율'].iloc[i])
    할인요소_보종['1-조정비율'] = 0
    for i in range(m):
        if i==0:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]
        else:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]-할인요소_보종['조정비율'].iloc[i-1]
    할인요소_보종['MATURITY'] = np.round(할인요소_보종['보험료_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소.append(할인요소_보종)
할인요소 = pd.concat(할인요소).reset_index(drop=True)
할인요소 = 할인요소.merge(자동차_분류, on='AT_PDGR_CD', how='left').drop('AT_PDGR_CD', axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Prog

In [89]:
# 할인요소 집계
보험료부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['지급보험금_비중']*df['DISCOUNT_x']).sum())
준비금부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['1-조정비율']*df['DISCOUNT_y']).sum())
자동차_할인요소_집계 = pd.concat([보험료부채_할인요소, 준비금부채_할인요소], axis=1)
자동차_할인요소_집계.columns = ['할인요소_보험료', '할인요소_준비금']
자동차_할인요소_집계 = 자동차_할인요소_집계.reset_index()
자동차_할인요소_집계

,PDGR_NM,할인요소_보험료,할인요소_준비금
0,개인용자동차(물담보),0.9886,0.9923
1,개인용자동차(인담보),0.9728,0.9780
2,업무용자동차(물담보),0.9882,0.9919
3,업무용자동차(인담보),0.9694,0.9748
4,영업용자동차(물담보),0.9867,0.9906
5,영업용자동차(인담보),0.9703,0.9757
6,자동차기타,0.9827,0.9871


#### 7-2. 재보험자산 손실조정

In [90]:
pd.options.display.float_format = '{:,.4f}'.format

In [91]:
선도부도율_가공 = 선도부도율 >> left_join(신용등급분류, on='CRD_GRD_CD') \
    >> mutate(YEAR=X.MAT_CD.str.slice(1).astype(int)//12) \
    >> drop(['MAT_CD', 'CRD_GRD_CD', 'BASE_YYMM']) \
    >> mask(X.YEAR <= 10)

In [92]:
# 준비금부채 손실조정
손실조정_준비금부채 = 할인요소 >> select(['PDGR_NM', 'YEAR', '준비금_평균지급기간', '지급보험금_비중', 'DISCOUNT_x']) \
    >> arrange(X.PDGR_NM, -X.준비금_평균지급기간) \
    >> mutate(손실예상액=X.지급보험금_비중*0.5)
손실조정_준비금부채.columns = ['PDGR_NM', 'YEAR', '준비금_평균지급기간', '지급보험금_비중', '할인율', '손실예상액']
부도시손실액 = 손실조정_준비금부채.groupby('PDGR_NM').apply(lambda df: pd.concat([df[['PDGR_NM', '준비금_평균지급기간']], df[['손실예상액']].cumsum()], axis=1))
부도시손실액.columns = ['PDGR_NM', '준비금_평균지급기간', '부도시손실액']
손실조정_준비금부채 = 손실조정_준비금부채 >> left_join(부도시손실액, on='PDGR_NM') \
    >> arrange(X.PDGR_NM, X.준비금_평균지급기간) \
    >> left_join(선도부도율_가공, on='YEAR') \
    >> mutate(준비금_손실조정=X.FWD_PD*X.부도시손실액*X.할인율) \
    >> group_by(X.PDGR_NM, X.GRADE) \
    >> summarise(준비금_손실조정=np.sum(X.준비금_손실조정))

In [93]:
# 준비금부채 손실조정
손실조정_보험료부채 = 할인요소 >> select(['PDGR_NM', 'YEAR', '보험료_평균지급기간', '1-조정비율', 'DISCOUNT_y']) \
    >> arrange(X.PDGR_NM, -X.보험료_평균지급기간)
손실조정_보험료부채['손실예상액'] = 손실조정_보험료부채['1-조정비율']*0.5
손실조정_보험료부채.columns = ['PDGR_NM', 'YEAR', '보험료_평균지급기간', '1-조정비율', '할인율', '손실예상액']
부도시손실액 = 손실조정_보험료부채.groupby('PDGR_NM').apply(lambda df: pd.concat([df[['PDGR_NM', '보험료_평균지급기간']], df[['손실예상액']].cumsum()], axis=1))
부도시손실액.columns = ['PDGR_NM', '보험료_평균지급기간', '부도시손실액']
손실조정_보험료부채 = 손실조정_보험료부채 >> left_join(부도시손실액, on='PDGR_NM') \
    >> arrange(X.PDGR_NM, X.보험료_평균지급기간) \
    >> left_join(선도부도율_가공, on='YEAR') \
    >> mutate(보험료_손실조정=X.FWD_PD*X.부도시손실액*X.할인율) \
    >> group_by(X.PDGR_NM, X.GRADE) \
    >> summarise(보험료_손실조정=np.sum(X.보험료_손실조정))

In [107]:
# 집계
자동차_손실조정_집계 = 손실조정_준비금부채.merge(손실조정_보험료부채, on=['PDGR_NM', 'GRADE'], how='outer')
자동차_손실조정_집계

,GRADE,PDGR_NM,준비금_손실조정,보험료_손실조정
0,A,개인용자동차(물담보),0.0024,0.0024
1,A+,개인용자동차(물담보),0.0015,0.0015
2,A-,개인용자동차(물담보),0.0039,0.0039
3,AA,개인용자동차(물담보),0.0006,0.0006
4,AA+,개인용자동차(물담보),0.0003,0.0003
5,AA-,개인용자동차(물담보),0.0010,0.0010
6,AAA,개인용자동차(물담보),0.0002,0.0002
7,B,개인용자동차(물담보),0.1092,0.1093
8,BB,개인용자동차(물담보),0.0261,0.0260
9,BB+,개인용자동차(물담보),0.0192,0.0192


#### 8. 자동차/출재(비례)/계약 - 직전 1년 경과보험료

In [108]:
# 일반 출재 계약 PDGR_NM 가공
자동차_출재_비례_직전1년경과보험료_가공 = 자동차_출재_비례_직전1년경과보험료 >> left_join(자동차_상품군정보, by=['PDGR_CD', 'BSC_CVR_CD']) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

In [109]:
# 일반 출재 계약 직전1년경과보험료 집계
자동차_출재_비례_직전1년경과보험료_집계 = 자동차_출재_비례_직전1년경과보험료_가공 >> group_by(X.PDGR_NM) \
    >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM).astype(float)) \
    >> mutate(RN_ELP_PRM=X.RN_ELP_PRM/1e3)
자동차_출재_비례_직전1년경과보험료_집계

,PDGR_NM,RN_ELP_PRM
0,개인용자동차(물담보),"4,373,066.7810"
1,개인용자동차(인담보),"2,170,734.1940"
2,업무용자동차(물담보),"4,813,128.1020"
3,업무용자동차(인담보),"3,344,851.7280"
4,영업용자동차(인담보),-95.9600
5,자동차기타,"1,095,674.9160"


#### 9 . 자동차/출재(비비례)/계약 - 직전 1년 경과보험료

In [110]:
자동차_출재_비비례_직전1년경과보험료_집계 = 자동차_출재_비비례_직전1년경과보험료 >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM).astype(float)) \
    >> mutate(RN_ELP_PRM=X.RN_ELP_PRM/1e3)
자동차_출재_비비례_직전1년경과보험료_집계

,RN_ELP_PRM
0,"3,328,550.6700"


#### 10. 자동차/원수/계약 - 직전 1년 경과보험료

In [111]:
# 자동차 출재 계약 연동수수료비중
자동차_출재_직전1년경과보험료_연동수수료비중 = 자동차_출재_비례_직전1년경과보험료_집계['RN_ELP_PRM'].sum()/(자동차_출재_비비례_직전1년경과보험료_집계['RN_ELP_PRM'].sum()+자동차_출재_비례_직전1년경과보험료_집계['RN_ELP_PRM'].sum())
자동차_출재_연동수수료비중_연동 = 자동차_출재_비례_직전1년경과보험료_집계[['PDGR_NM']]
자동차_출재_연동수수료비중_연동['CMSN_DVCD'] = '연동'
자동차_출재_연동수수료비중_연동['WEIGHT'] = 자동차_출재_직전1년경과보험료_연동수수료비중
자동차_출재_연동수수료비중_비연동 = 자동차_출재_비례_직전1년경과보험료_집계[['PDGR_NM']]
자동차_출재_연동수수료비중_비연동['CMSN_DVCD'] = '비연동'
자동차_출재_연동수수료비중_비연동['WEIGHT'] = 1-자동차_출재_직전1년경과보험료_연동수수료비중
자동차_출재_직전1년경과보험료_연동수수료비중 = pd.concat([자동차_출재_연동수수료비중_연동, 자동차_출재_연동수수료비중_비연동]).reset_index(drop=True)

In [112]:
# PDGR_NM 가공
자동차_원수_직전1년경과보험료_가공 = 자동차_원수_직전1년경과보험료 >> left_join(자동차_상품군정보, on=[X.PDGR_CD, X.BSC_CVR_CD]) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isna(), '자동차기타', X.PDGR_NM))

# 자동차 원수 직전1년경과보험료 배분 (1)
자동차_원수_직전1년경과보험료_배분1 = 자동차_원수_직전1년경과보험료_가공 >> mask(X.RN_DVCD=='X') \
    >> mutate(CMSN_DVCD='비연동')

# 자동차 원수 직전1년경과보험료 배분 (2)
자동차_원수_직전1년경과보험료_배분2 = 자동차_원수_직전1년경과보험료_가공 >> mask(X.RN_DVCD=='O') \
    >> left_join(자동차_출재_직전1년경과보험료_연동수수료비중, on='PDGR_NM') \
    >> mutate(ELP_PRM=X.ELP_PRM*X.WEIGHT)

In [113]:
# 자동차 원수 직전1년경과보험료 집계
자동차_원수_직전1년경과보험료_집계1 = 자동차_원수_직전1년경과보험료_배분1 >> group_by(X.PDGR_NM, X.CMSN_DVCD) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM).astype(float)) \
    >> mutate(ELP_PRM=X.ELP_PRM/1e3)

자동차_원수_직전1년경과보험료_집계2 = 자동차_원수_직전1년경과보험료_배분2 >> group_by(X.PDGR_NM, X.CMSN_DVCD) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM)) \
    >> mutate(ELP_PRM=X.ELP_PRM/1e3)

자동차_원수_직전1년경과보험료_집계 = pd.concat([자동차_원수_직전1년경과보험료_집계1, 자동차_원수_직전1년경과보험료_집계2]) \
    >> group_by(X.PDGR_NM, X.CMSN_DVCD) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM))
자동차_원수_직전1년경과보험료_집계

,CMSN_DVCD,PDGR_NM,ELP_PRM
0,비연동,개인용자동차(물담보),"1,248,727,499.5993"
1,연동,개인용자동차(물담보),"24,067,944.7607"
2,비연동,개인용자동차(인담보),"731,469,363.6531"
3,연동,개인용자동차(인담보),"11,946,362.3989"
4,비연동,업무용자동차(물담보),"428,544,376.3843"
5,연동,업무용자동차(물담보),"13,251,099.7457"
6,비연동,업무용자동차(인담보),"274,222,910.4743"
7,연동,업무용자동차(인담보),"9,208,674.5907"
8,비연동,영업용자동차(물담보),"37,719,578.5500"
9,비연동,영업용자동차(인담보),"105,441,972.6509"


#### 11. 자동차/출재(비례)/보상 - 개별추산액 (2)

In [114]:
# 자동차 출재 보상 연동수수료비중
자동차_출재_개별추산액_연동수수료비중 = 자동차_출재_비례_개별추산액_집계['RN_OST_AMT'].sum()/(자동차_출재_비비례_개별추산액_집계['RN_OST_AMT'].sum()+자동차_출재_비례_개별추산액_집계['RN_OST_AMT'].sum())
자동차_출재_연동수수료비중_연동 = 자동차_출재_비례_개별추산액_집계[['PDGR_NM']]
자동차_출재_연동수수료비중_연동['CMSN_DVCD'] = '연동'
자동차_출재_연동수수료비중_연동['WEIGHT'] = 자동차_출재_개별추산액_연동수수료비중
자동차_출재_연동수수료비중_비연동 = 자동차_출재_비례_개별추산액_집계[['PDGR_NM']]
자동차_출재_연동수수료비중_비연동['CMSN_DVCD'] = '비연동'
자동차_출재_연동수수료비중_비연동['WEIGHT'] = 1-자동차_출재_개별추산액_연동수수료비중
자동차_출재_개별추산액_연동수수료비중 = pd.concat([자동차_출재_연동수수료비중_연동, 자동차_출재_연동수수료비중_비연동]).reset_index(drop=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [115]:
# 자동차 원수 개별추산액2 배분 (1)
자동차_원수_개별추산액_배분1 = 자동차_원수_개별추산액_가공 >> mask(X.RN_DVCD=='X') \
    >> mutate(CMSN_DVCD='비연동')

# 자동차 원수 개별추산액2 배분 (2)
자동차_원수_개별추산액_배분2 = 자동차_원수_개별추산액_가공 >> mask(X.RN_DVCD=='O') \
    >> left_join(자동차_출재_개별추산액_연동수수료비중, on='PDGR_NM') \
    >> mutate(OST_AMT=X.OST_AMT*X.WEIGHT)

In [116]:
# 자동차 원수 개별추산액 집계
자동차_원수_개별추산액_집계1 = 자동차_원수_개별추산액_배분1 >> group_by(X.PDGR_NM, X.CMSN_DVCD) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT).astype(float)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)

자동차_원수_개별추산액_집계2 = 자동차_원수_개별추산액_배분2 >> group_by(X.PDGR_NM, X.CMSN_DVCD) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)

자동차_원수_개별추산액2_집계 = pd.concat([자동차_원수_개별추산액_집계1, 자동차_원수_개별추산액_집계2]) \
    >> group_by(X.PDGR_NM, X.CMSN_DVCD) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT))
자동차_원수_개별추산액2_집계

,CMSN_DVCD,PDGR_NM,OST_AMT
0,비연동,개인용자동차(물담보),"70,651,306.7341"
1,연동,개인용자동차(물담보),"570,537.4949"
2,비연동,개인용자동차(인담보),"256,709,000.1499"
3,연동,개인용자동차(인담보),"4,637,646.6411"
4,비연동,업무용자동차(물담보),"20,619,004.6078"
5,연동,업무용자동차(물담보),"1,516,572.9802"
6,비연동,업무용자동차(인담보),"106,050,290.6313"
7,연동,업무용자동차(인담보),"2,865,908.1707"
8,비연동,영업용자동차(물담보),"2,290,806.0580"
9,비연동,영업용자동차(인담보),"46,563,209.8933"


In [117]:
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
now = datetime.now().strftime('%Y%m%d%H%M%S')
ver = 'v1.1.0'
with pd.ExcelWriter('./result/자동차_현행추정부채_{}_{}.xlsx'.format(ver, now)) as writer:
    자동차_원수_미경과보험료_집계.to_excel(writer, '자동차_원수_미경과보험료', index=False)
    자동차_출재_비례_미경과보험료_집계.to_excel(writer, '자동차_출재_비례_미경과보험료', index=False)
    자동차_원수_IBNR_집계.to_excel(writer, '자동차_원수_IBNR', index=False)
    자동차_출재_비례_IBNR_집계.to_excel(writer, '자동차_출재_비례_IBNR', index=False)
    자동차_원수_개별추산액_집계.to_excel(writer, '자동차_원수_개별추산액', index=False)
    자동차_원수_개별추산액2_집계.to_excel(writer, '자동차_원수_개별추산액2', index=False)
    자동차_출재_비례_개별추산액_집계.to_excel(writer, '자동차_출재_비례_개별추산액', index=False)
    자동차_출재_비비례_개별추산액_집계.to_excel(writer, '자동차_출재_비비례_개별추산액', index=False)
    자동차_납입예정보험료_집계.to_excel(writer, '자동차_납입예정보험료', index=False)
    자동차_할인요소_집계.to_excel(writer, '자동차_할인요소', index=False)
    자동차_손실조정_집계.to_excel(writer, '자동차_손실조정', index=False)
    자동차_원수_직전1년경과보험료_집계.to_excel(writer, '자동차_원수_직전1년경과보험료', index=False)
    자동차_출재_비례_직전1년경과보험료_집계.to_excel(writer, '자동차_출재_비례_직전1년경과보험료', index=False)
    자동차_출재_비비례_직전1년경과보험료_집계.to_excel(writer, '자동차_출재_비비례_직전1년경과보험료', index=False)
    자동차_원수_손해조사비율_집계.to_excel(writer, '자동차_원수_손해조사비율', index=False)